In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

get_ipython().magic('matplotlib inline')
sns.set(style='white', font_scale=0.9)
flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
sns.color_palette(flatui)

np.set_printoptions(threshold=np.nan)
pd.set_option("display.max_columns",100)

In [5]:
dataset = pd.read_csv('HW2_pokemon.csv')

In [6]:
dataset.head(10)

,Unnamed: 0,Name,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Color,Height_m,Weight_kg,Body_Style
0,0,pokemon1,309,39,52,43,60,50,65,Red,0.61,8.5,bipedal_tailed
1,1,pokemon2,405,58,64,58,80,65,80,Red,1.09,19.0,bipedal_tailed
2,2,pokemon3,314,44,48,65,50,64,43,Blue,0.51,9.0,bipedal_tailed
3,3,pokemon4,405,59,63,80,65,80,58,Blue,0.99,22.5,bipedal_tailed
4,4,pokemon5,530,79,83,100,85,105,78,Blue,1.60,85.5,bipedal_tailed
5,5,pokemon6,195,45,30,35,20,20,45,Green,0.30,2.9,insectoid
6,6,pokemon7,205,50,20,55,25,25,30,Green,0.71,9.9,serpentine_body
7,7,pokemon8,253,30,56,35,25,35,72,Purple,0.30,3.5,quadruped
8,8,pokemon9,413,55,81,60,50,70,97,Brown,0.71,18.5,quadruped
9,9,pokemon10,288,35,60,44,40,54,55,Purple,2.01,6.9,serpentine_body


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 13 columns):
Unnamed: 0    368 non-null int64
Name          368 non-null object
Total         368 non-null int64
HP            368 non-null int64
Attack        368 non-null int64
Defense       368 non-null int64
Sp_Atk        368 non-null int64
Sp_Def        368 non-null int64
Speed         368 non-null int64
Color         368 non-null object
Height_m      368 non-null float64
Weight_kg     368 non-null float64
Body_Style    368 non-null object
dtypes: float64(2), int64(8), object(3)
memory usage: 37.5+ KB


In [8]:
dataset['Type 2'] = dataset['Type 2'].fillna('None')

KeyError: 'Type 2'

In [ ]:
dataset_num = dataset[['Total','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
dataset_num = sc_X.fit_transform(dataset_num)

In [ ]:

dataset_scaleddataset  = dataset.copy()
dataset_scaled[['Total','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']] = dataset_num

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 41):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(dataset_num)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 41), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.xticks(np.arange(1, 41, 1.0))
plt.grid(which='major', axis='x')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(dataset_num)

In [ ]:
dataset['y_kmeans'] = y_kmeans

In [ ]:
dataset.head()

In [ ]:
sns.violinplot(x='y_kmeans', y='Total', data=dataset)
plt.show()

In [ ]:
sns.violinplot(x='y_kmeans', y='Attack', data=dataset)
plt.show()

In [ ]:
sns.violinplot(x='y_kmeans', y='Defense', data=dataset)
plt.show()

In [ ]:
dataset.sort_values('Defense', axis=0, ascending=False).head(10)

In [ ]:
sns.violinplot(x='y_kmeans', y='Speed', data=dataset)
plt.show()

In [ ]:
dataset.sort_values('Speed', axis=0, ascending=False).head(15)

In [ ]:
sns.violinplot(x='y_kmeans', y='Sp. Atk', data=dataset)
plt.show()

In [ ]:
sns.violinplot(x='y_kmeans', y='Sp. Def', data=dataset)
plt.show()

In [ ]:

datasetdataset..sort_valuessort_val ('Sp. Def', axis=0, ascending=False).head(10)

In [ ]:
# Clusters by Type

#Get counts by type and cluster
#We need to merge the two columns Type 1 and Type 2 together the type can appear in either column
data_pct_1 = dataset.groupby(['Type 1', 'y_kmeans'])['Name'].count().to_frame().reset_index()
data_pct_1.columns = ['Type', 'y_kmeans', 'count_1']

data_pct_2 = dataset.groupby(['Type 2', 'y_kmeans'])['Name'].count().to_frame().reset_index()
data_pct_2.columns = ['Type', 'y_kmeans', 'count_2']

data_pct = data_pct_1.merge(data_pct_2, how='outer',
                            left_on=['Type', 'y_kmeans'],
                            right_on=['Type', 'y_kmeans'])

data_pct.fillna(0, inplace=True)
data_pct['count'] = data_pct['count_1'] + data_pct['count_2']

#Get counts by type
data_pct_Total = data_pct.groupby(['Type']).sum()['count'].reset_index()
data_pct_Total.columns = ['Type', 'count_total']

#Merge two dataframes and create percentage column
data_pct = data_pct.merge(right=data_pct_Total, 
                                    how='inner',
                                    left_on='Type',
                                    right_on='Type')

data_pct['pct'] = data_pct['count'] / data_pct['count_total']

#Create Graph
sns.barplot(x='Type', y='pct', data=data_pct, estimator=sum, ci=None, color='#34495e', label='4')
sns.barplot(x='Type', y='pct', data=data_pct[data_pct['y_kmeans'] <= 3], 
            estimator=sum, ci=None, color='#e74c3c', label='3') 
sns.barplot(x='Type', y='pct', data=data_pct[data_pct['y_kmeans'] <= 2], 
            estimator=sum, ci=None, color='#95a5a6', label='2') 
sns.barplot(x='Type', y='pct', data=data_pct[data_pct['y_kmeans'] <= 1], 
            estimator=sum, ci=None, color='#3498db', label='1') 
sns.barplot(x='Type', y='pct', data=data_pct[data_pct['y_kmeans'] == 0], 
            estimator=sum, ci=None, color='#9b59b6', label='0') 

plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1))
plt.xticks(rotation=90)
plt.ylabel('Percentage')
plt.tight_layout()
plt.show()

In [ ]:
#Clusters by Generation


#Get counts by generation and cluster
#We need to merge the two columns Type 1 and Type 2 together the type can appear in either column
data_pct = dataset.groupby(['Generation', 'y_kmeans'])['Name'].count().to_frame().reset_index()
data_pct.columns = ['Generation', 'y_kmeans', 'count']

#Get counts by type
data_pct_Total = data_pct.groupby(['Generation']).sum()['count'].reset_index()
data_pct_Total.columns = ['Generation', 'count_total']

#Merge two dataframes and create percentage column
data_pct = data_pct.merge(right=data_pct_Total, 
                                    how='inner',
                                    left_on='Generation',
                                    right_on='Generation')

data_pct['pct'] = data_pct['count'] / data_pct['count_total']

#Create Graph
sns.barplot(x='Generation', y='pct', data=data_pct, estimator=sum, ci=None, color='#34495e', label='4')
sns.barplot(x='Generation', y='pct', data=data_pct[data_pct['y_kmeans'] <= 3], 
            estimator=sum, ci=None, color='#e74c3c', label='3') 
sns.barplot(x='Generation', y='pct', data=data_pct[data_pct['y_kmeans'] <= 2], 
            estimator=sum, ci=None, color='#95a5a6', label='2') 
sns.barplot(x='Generation', y='pct', data=data_pct[data_pct['y_kmeans'] <= 1], 
            estimator=sum, ci=None, color='#3498db', label='1') 
sns.barplot(x='Generation', y='pct', data=data_pct[data_pct['y_kmeans'] == 0], 
            estimator=sum, ci=None, color='#9b59b6', label='0') 

plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1))
plt.xticks(rotation=90)
plt.ylabel('Percentage')
plt.tight_layout()
plt.show()